In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_train = pd.read_csv('../input/big-mart-sales-prediction/Train.csv')
df_test = pd.read_csv('../input/big-mart-sales-prediction/Test.csv')

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df1 = df_train

In [ ]:
df1.describe()

In [ ]:
#columns with null values
df1.isnull().sum()

In [ ]:
df1.columns

In [ ]:
categorical = []
for i in df1.dtypes.index:
    if (df1.dtypes[i] == 'object'):
        categorical.append(i)
print(categorical)

In [ ]:
df1.dtypes['Item_Visibility'] == 'object'

In [ ]:
print(len(df1['Item_Identifier'].unique()))
print(len(df1))

In [ ]:
weight_mean = pd.DataFrame(df1.groupby('Item_Identifier')['Item_Weight'].mean())
weight_mean

In [ ]:
missing = df1['Item_Weight'].isnull()
missing

In [ ]:
#for item in df1['Item_Identifier']:
    #print(item)

In [ ]:
#for i,item in enumerate(df1['Item_Identifier']):
    #print(i,item)

In [ ]:
for i,item in enumerate(df1['Item_Identifier']):
    if (missing[i] == True):
        if item is weight_mean:
            df1['Item_Weight'][i] = weight_mean.loc['Item_Identifier']['Item_Weight']
        else:
            df1['Item_Weight'][i] = np.mean(df1['Item_Weight'])

In [ ]:
#for i in df1['Item_Identifier']:
    #print(i)
df1['Item_Weight'].isnull().sum()

In [ ]:
df1['Outlet_Type'].unique()

In [ ]:
df1['Outlet_Size'].unique()

In [ ]:
size_mode = df1.pivot_table(values = 'Outlet_Size' , columns = 'Outlet_Type' , 
                            aggfunc = (lambda x:x.mode()))
size_mode

In [ ]:
size_mode['Grocery Store']

In [ ]:
miss_size = df1['Outlet_Size'].isnull()
miss_size

In [ ]:
def Outlet_size_replace(x):
    return size_mode[x]

In [ ]:
Outlet_size_replace('Supermarket Type2')[0]

In [ ]:
#df1['Outlet_Type'].apply(Outlet_size_replace)

In [ ]:
#.apply() : to apply a function in a data column
#df1.loc[miss_size]['Outlet_Type'].apply(Outlet_size_replace)

In [ ]:
mvd = df1.loc[miss_size]
nvd = df1.loc[miss_size != True]

In [ ]:
mvd['Outlet_Size'] = mvd['Outlet_Type'].apply(Outlet_size_replace)

In [ ]:
df2 = pd.concat((mvd,nvd),ignore_index=True)

In [ ]:
df2.isnull().sum()

In [ ]:
df2.head()

In [ ]:
df2[df2['Item_Visibility'] == 0]

In [ ]:
df2['Item_Visibility'].mean()

In [ ]:
df2['Item_Visibility'] = df2['Item_Visibility'].replace(0,df2['Item_Visibility'].mean())

In [ ]:
df2[df2['Item_Visibility'] == 0]

In [ ]:
df2.head()

In [ ]:
#value_counts()
df2['Item_Fat_Content'].value_counts()

In [ ]:
df2['Item_Fat_Content'] = df2['Item_Fat_Content'].replace({'LF' : 'Low Fat' , 
                                                           'low fat' : 'Low Fat' , 
                                                           'reg' : 'Regular'})

In [ ]:
df2['Item_Fat_Content'].value_counts()

In [ ]:
df2.head()

In [ ]:
len(df2['Item_Identifier'].unique())

In [ ]:
k = 'FDU28'
k[:2]

In [ ]:
def identifyItem(x):
    return (x[:2])

identifyItem('NCD06')

In [ ]:
df2['Item_Identifier'] = df2['Item_Identifier'].apply(identifyItem)

In [ ]:
df2.head()

In [ ]:
len(df2['Item_Identifier'].unique())

In [ ]:
df2['Item_Identifier'].unique()

In [ ]:
df2['Item_Identifier'] = df2['Item_Identifier'].replace({'FD':'Food', 
                                                         'NC':'Non-Consumable', 
                                                         'DR':'Drink'})

In [ ]:
df2.head()

In [ ]:
df2.loc[df2['Item_Identifier'] == 'Non-Consumable' , 'Item_Fat_Content']

In [ ]:
df2.loc[df2['Item_Identifier'] == 'Non-Consumable' , 'Item_Fat_Content'] = 'Non-Edible'

In [ ]:
df2['Item_Fat_Content'].value_counts()

In [ ]:
df2.head()

In [ ]:
df2['Years'] = 2020 - df2['Outlet_Establishment_Year']
df2.head()

In [ ]:
#Data Analysis by plotting

In [ ]:
sns.distplot(df2['Item_Weight'])

In [ ]:
sns.distplot(df2['Item_Visibility'])

In [ ]:
sns.distplot(df2['Item_MRP'])

In [ ]:
sns.distplot(df2['Item_Outlet_Sales'])

In [ ]:
df2['Item_Outlet_Sales'] = np.log(1+df2['Item_Outlet_Sales'])

In [ ]:
sns.distplot(df2['Item_Outlet_Sales'])

In [ ]:
sns.countplot(df2['Item_Fat_Content'])

In [ ]:
df2['Item_Type'].unique()

In [ ]:
myplot = sns.countplot(df2['Item_Type'])
myplot.set_xticklabels(labels = df2['Item_Type'].unique() , rotation=90)

In [ ]:
sns.countplot(df2['Outlet_Establishment_Year'])

In [ ]:
sns.countplot(df2['Outlet_Size'])

In [ ]:
sns.countplot(df2['Outlet_Location_Type'])

In [ ]:
myplot = sns.countplot(df2['Outlet_Type'])
myplot.set_xticklabels(labels = df2['Outlet_Type'].unique() , rotation=90)

In [ ]:
df2.head()

In [ ]:
categorical = []
for i in df1.dtypes.index:
    if (df1.dtypes[i] == 'object'):
        categorical.append(i)
print(categorical)

In [ ]:
len(df2['Item_Fat_Content'].unique())

In [ ]:
for i in categorical:
    print(i , len(df2[i].unique()))

In [ ]:
Item_count = df2['Item_Type'].value_counts()
Item_count_less = Item_count[Item_count <= 251]
Item_count_less

In [ ]:
'Breakfast' in Item_count_less

In [ ]:
def createOthers(x):
    if x in Item_count_less:
        return 'Others'
    else:
        return x

In [ ]:
createOthers('Seafood')

In [ ]:
df2['Item_Type'] = df2['Item_Type'].apply(createOthers)

In [ ]:
df2['Item_Type'].value_counts()

In [ ]:
for i in categorical:
    print(i , len(df2[i].unique()))

In [ ]:
df2.head()
df3 = df2.drop(['Outlet_Identifier','Outlet_Establishment_Year'],axis=1)

In [ ]:
df3.head()

In [ ]:
df3.dtypes

#One-hot encoding

In [ ]:
df3['Item_Identifier'].unique()

In [ ]:
#Food-0,Non-Consumable-1,Drink-2
#Food -           1 0 0
#Non-Consumable - 0 1 0
#Drink -          0 0 1

In [ ]:
df_item_identifier = pd.get_dummies(df3['Item_Identifier'])
df_fat = pd.get_dummies(df3['Item_Fat_Content'])
df_outlet_size = pd.get_dummies(df3['Outlet_Size'])
df_outlet_location = pd.get_dummies(df3['Outlet_Location_Type'])
df_outlet_type = pd.get_dummies(df3['Outlet_Type'])
df_item_type = pd.get_dummies(df3['Item_Type'])

In [ ]:
df4 = pd.concat([df3,df_fat,df_outlet_size,df_outlet_location,
                 df_outlet_type,df_item_type,df_item_identifier],axis='columns')

In [ ]:
df4.head()

In [ ]:
df5 = df4.drop(['Item_Identifier','Item_Fat_Content','Item_Type','Outlet_Size','Outlet_Location_Type','Outlet_Type'],axis=1)

In [ ]:
df5.head()

In [ ]:
df5.dtypes

In [ ]:
X = df5.drop('Item_Outlet_Sales' , axis=1)
Y = df5['Item_Outlet_Sales']

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X,Y,train_size=0.8)

In [ ]:
from sklearn.linear_model import LinearRegression
lmodel = LinearRegression(normalize=True)

In [ ]:
lmodel.fit(xtrain,ytrain)

In [ ]:
ypred_train = lmodel.predict(xtrain)
ypred_test = lmodel.predict(xtest)

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error
print("Training Mean Absolute Error",mean_absolute_error(ytrain,ypred_train))
print("Testing Mean Absolute Error",mean_absolute_error(ytest,ypred_test))
      

In [ ]:
Y.min()
Y.max()

In [ ]:
coef = pd.Series(lmodel.coef_,xtrain.columns).sort_values()
coef.plot(kind='bar' , title="Model Coefficients")

In [ ]:
print(xtrain.columns)

In [ ]:
xtrain.shape[1]

In [ ]:
df3.columns

In [ ]:
np.where(xtrain.columns == 'Non-Edible')[0][0]

In [ ]:
def predict_sales(Item_Weight,Item_Visibility,Item_MRP,Years,Item_Fat_Content,Item_Type,
                 Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Identifier):

    uv = np.zeros(xtrain.shape[1])
    uv[0] = Item_Weight
    uv[1] = Item_Visibility
    uv[2] = Item_MRP
    uv[3] = Years

    uv[np.where(xtrain.columns == Item_Fat_Content)[0][0]] = 1
    uv[np.where(xtrain.columns == Item_Type)[0][0]] = 1
    uv[np.where(xtrain.columns == Outlet_Size)[0][0]] = 1
    uv[np.where(xtrain.columns == Outlet_Location_Type)[0][0]] = 1
    uv[np.where(xtrain.columns == Outlet_Type)[0][0]] = 1
    uv[np.where(xtrain.columns == Item_Identifier)[0][0]] = 1

    return lmodel.predict([uv])

In [ ]:
print(predict_sales(9.30,0.03,250.20,20,
                    'Low Fat','Dairy','Medium','Tier 1','Supermarket Type2','Food'))

In [ ]:
#pickle
import pickle
with open('sales_value.pickle','wb') as f:
    pickle.dump(lmodel,f)

In [ ]:
import json
columns = {'data_columns' : [col.lower() for col in xtrain.columns]}

with open('columns.json','w') as k:
    k.write(json.dumps(columns))